## ModelFront API with Python

This notebook shows you how to call the **[ModelFront](https://modelfront.com) API** with Python.

**ModelFront machine translation risk prediction** instantly predicts if a translation is correct or not.

Not all topics and languages are supported yet.  To request a language, email languages@modelfront.com.  For other questions, email support@modelfront.com.

Not a Pythonista?  To call ModelFront from bash directly, use [api.modelfront.sh](https://modelfront.com/api.modelfront.sh) instead.

## Getting started

We assume that you have a ModelFront API key and are using Python 3.  To get a ModelFront API key, sign up at [modelfront.com](https://modelfront.com).

To get started with the API, save your own copy of this notebook and replace `YOURKEYHERE` with your key, and hit *Run*.

In [0]:
%env MODELFRONT_KEY=YOURKEYHERE

! pip3 install requests

import os, requests

key = os.environ['MODELFRONT_KEY']

def send(sl, tl, data):
  url = f'https://api.modelfront.com/v1?sl={sl}&tl={tl}&token={key}'
  r = requests.post(url, json=data)
  if r.json()['status'] == 'error':
    print(r.json()['status'])
    return
  risks = r.json()['risks']
  for i, row in enumerate(data['rows']):
    original = row['original']
    translation = row['translation']
    risk = risks[i]
    print(f'{risk}  \t {sl}: {original}  {tl}: {translation}')


def run(sl, tl, original, translation):
  data = {
    'meta': {},
    'rows': [
      { 'original': original, 'translation': translation}
    ]
  }
  send(sl, tl, data)


def run_batch(sl, tl, tsv_text):
  rows = []
  for line in tsv_text.strip().split('\n'):
    [ original, translation ] = line.split('\t')
    rows += [ { 'original': original, 'translation': translation } ]
  data = {
    'meta':{},
    'rows': rows
  }
  send(sl, tl, data)


## Sending requests

Now you can send a request to get risk prediction for a translation.

`sl` is the source language, `tl` is the target language, `original` is the source text and `translation` is the target text.  Each text should be a single sentene or less, and can be up to 300 characters long.

For example, to test English to Portuguese, run the following:

In [0]:
run('en', 'pt', 'This is not a test.', 'Isto não é uma prova.')

A risk greater than 0.5 (50%) implies that the translation is probably wrong.

Now try a bad translation of the same text:

In [0]:
run('en', 'pt', 'This is not a test.', 'Isto é uma prova.')

The risk should now be significantly higher.

### Batch requests

To request risk predictions for a batch, use `run_batch`.  You can include up to 15 translations in one request.

Note that there are hidden tab characters separating each sentence pair below.  You can also `\t` explicitly, or change `run_batch` to support another format.

In [0]:
run_batch('en', 'pt', 
"""
This is not a test.	Isto não é uma prova.
This is not a test.	Isto é uma prova.
Bose wireless	Sem fio Bose
Down Alternatives	Down Alternativas
3" Bull Bar $99	3 "Bull Bar $ 99
Phone case	Capa de celular
Phone case used	Caso de telefone usado
Manhattan	Manhattan
Long Island	Ilha Longa
Long Island	Long Island
Mountain View	vista da montanha
Mountain View	Mountain View
""")

Questions, comments or feature requests?  [Let us know!](mailto:feedback@modelfront.com)